In [1]:
import pandas as pd
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gensim
from collections import defaultdict
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# read in the data using Pandas
data = pd.read_csv('globe_data/bostonglobe2018.csv')

In [3]:
import nltk

stemmer = nltk.stem.SnowballStemmer('english')

# strip the text of unnecessary whitespaces and removing special characters
def custom_standardization(data):

    spec_chars = ["!",'"',"#","%","&","'","(",")", "*","+",",",
                  "-",".","/",":",";","<", "=",">","?","@","[",
                  "\\","]","^","_", "`","{","|","}","~","–", 
                  "\xc2", "\xa0", "\x80", "\x9c", "\x99", "\x94", 
                  "\xad", "\xe2", "\x9d", "\n"]

    for char in spec_chars:
        data['text'] = data['text'].str.strip()
        #data['text'] = str(data['text']).lower()
        data['text'] = data['text'].str.replace(char, ' ')
        #data['text'] = stemmer.stem(str(data['text']))
        
    return data

# do stemming before training
# compare differences between word vectors for 'black' as well as 'white' to better identify biased representations
# think about word frequency, so that context isn't specific, but rather, general
# increase data point size for visualizations

# instead of keywords, use sub-neighborhood names

# cluster similar words
# visualize those with a meta word that's representative
# either use average word vectors or minimizing function to get representative word

In [4]:
df = custom_standardization(data)

In [5]:
# turn DataFrame into a list of lists of tokens
documents = []
mentions = {'hyde_park': 0, 'beacon_hill': 0, 'south_boston': 0, 'jamaica_plain': 0, 'east_boston': 0,
           'south_end': 0, 'back_bay': 0, 'north_end': 0, 'west_roxbury': 0, 'mission_hill': 0,
           'harbor_islands': 0, 'west_end': 0, 'south_boston_waterfront': 0, 'longwood_medical_area': 0,
           'dorchester': 0, 'roxbury': 0, 'downtown': 0, 'fenway': 0, 'mattapan': 0, 'brighton': 0,
           'charlestown': 0, 'roslindale': 0, 'allston': 0}
for row in df.values:
    [row] = row
    temp = row.lower().split()
    length = len(temp) - 1
    for i in range(length):
        if temp[i] == 'hyde' and temp[i + 1] == 'park':
            temp[i] = 'hyde_park'
            temp.pop(i + 1)
            length -= 1
            mentions['hyde_park'] += 1
        elif temp[i] == 'dorchester':
            mentions['dorchester'] += 1
        elif temp[i] == 'roxbury':
            mentions['roxbury'] += 1
        elif temp[i] == 'downtown':
            mentions['downtown'] += 1
        elif temp[i] == 'fenway':
            mentions['fenway'] += 1
        elif temp[i] == 'mattapan':
            mentions['mattapan'] += 1
        elif temp[i] == 'brighton':
            mentions['brighton'] += 1
        elif temp[i] == 'charlestown':
            mentions['charlestown'] += 1
        elif temp[i] == 'roslindale':
            mentions['roslindale'] += 1
        elif temp[i] == 'allston':
            mentions['allston'] += 1
        elif temp[i] == 'beacon' and temp[i + 1] == 'hill':
            temp[i] = 'beacon_hill'
            temp.pop(i + 1)
            length -= 1
            mentions['beacon_hill'] += 1
        elif temp[i] == 'south' and temp[i + 1] == 'boston':
            temp[i] = 'south_boston'
            temp.pop(i + 1)
            length -= 1
            mentions['south_boston'] += 1
        elif temp[i] == 'jamaica' and temp[i + 1] == 'plain':
            temp[i] = 'jamaica_plain'
            temp.pop(i + 1)
            length -= 1
            mentions['jamaica_plain'] += 1
        elif temp[i] == 'east' and temp[i + 1] == 'boston':
            temp[i] = 'east_boston'
            temp.pop(i + 1)
            length -= 1
            mentions['east_boston'] += 1
        elif temp[i] == 'south' and temp[i + 1] == 'end':
            temp[i] = 'south_end'
            temp.pop(i + 1)
            length -= 1
            mentions['south_end'] += 1
        elif temp[i] == 'back' and temp[i + 1] == 'bay':
            temp[i] = 'back_bay'
            temp.pop(i + 1)
            length -= 1
            mentions['back_bay'] += 1
        elif temp[i] == 'north' and temp[i + 1] == 'end':
            temp[i] = 'north_end'
            temp.pop(i + 1)
            length -= 1
            mentions['north_end'] += 1
        elif temp[i] == 'west' and temp[i + 1] == 'roxbury':
            temp[i] = 'west_roxbury'
            temp.pop(i + 1)
            length -= 1
            mentions['west_roxbury'] += 1
        elif temp[i] == 'mission' and temp[i + 1] == 'hill':
            temp[i] = 'mission_hill'
            temp.pop(i + 1)
            length -= 1
            mentions['mission_hill'] += 1
        elif temp[i] == 'harbor' and temp[i + 1] == 'islands':
            temp[i] = 'harbor_islands'
            temp.pop(i + 1)
            length -= 1
            mentions['harbor_islands'] += 1
        elif temp[i] == 'west' and temp[i + 1] == 'end':
            temp[i] = 'west_end'
            temp.pop(i + 1)
            length -= 1
            mentions['west_end'] += 1
        elif temp[i] == 'cape' and temp[i + 1] == 'verdean':
            temp[i] = 'cape_verdean'
            temp.pop(i + 1)
            length -= 1
        elif temp[i] == 'afro' and temp[i + 1] == 'latino':
            temp[i] = 'afro_latino'
            temp.pop(i + 1)
            length -= 1
        elif temp[i] == 'afro' and temp[i + 1] == 'latina':
            temp[i] = 'afro_latina'
            temp.pop(i + 1)
            length -= 1
        elif temp[i] == 'african' and temp[i + 1] == 'american':
            temp[i] = 'african_american'
            temp.pop(i + 1)
            length -= 1
        elif temp[i] == 'west' and temp[i + 1] == 'indian':
            temp[i] = 'west_indian'
            temp.pop(i + 1)
            length -= 1            
        elif i > 0 and temp[i - 1] == 'south' and temp[i] == 'boston' and temp[i + 1] == 'waterfront':
            temp[i - 1] = 'south_boston_waterfront'
            temp.pop(i + 1)
            temp.pop(i)
            length -= 2
            mentions['south_boston_waterfront'] += 1
        elif i > 0 and temp[i - 1] == 'longwood' and temp[i] == 'medical' and temp[i + 1] == 'area':
            temp[i - 1] = 'longwood_medical_area'
            temp.pop(i + 1)
            temp.pop(i)
            length -= 1
            mentions['longwood_medical_area'] += 1
        elif i >= length - 3:
            break
    documents.append(temp)
    #print(len(documents))

In [6]:
# create Word2Vec model
# the skip-grams method is used here, with a window of 10
model = gensim.models.Word2Vec(window=10, min_count=2, sg=1, workers=10)
model.build_vocab(documents)  # prepare the model vocabulary

In [7]:
# train model on available data
# I use 5 epochs since that's standard
model.train(corpus_iterable=documents, total_examples=len(documents), epochs=5)

(27247293, 34085675)

In [8]:
black_neighborhoods = ['dorchester', 'roxbury', 'mattapan', 'hyde_park']
white_neighborhoods = ['fenway', 'beacon_hill', 'downtown', 'south_boston', 'east_boston', 'back_bay', 'jamaica_plain',
                      'south_end', 'charlestown', 'brighton', 'allston', 'west_end', 'roslindale', 'north_end',
                      'mission_hill', 
                       #'south_boston_waterfront', 
                       'harbor_islands', 'longwood_medical_area']

keywords = black_neighborhoods + white_neighborhoods

black_ethnicities = ['black', 'cape_verdean', #'afro_latino', 'afro_latina', 
                     'haitian', 'african_american', 
                     #'african-american', 
                     'caribbean', 'jamaican', 'dominican'] #, 'west_indian']

# extract vector for neighborhood
# vector for corresponding majority ethnicity
# add those together
# look for words that are close to that sum vector
# repeat for other ethnicities
# what differences do we get in terms of results for last week vs results for this week

In [9]:
tuples = []
for sub in black_neighborhoods:
    for ethnicity in black_ethnicities:
        tuples.append((ethnicity, sub))

In [10]:
#df['original_tuple'] = 'unknown'

# finding similar words and creating a csv file
# check if similar words list includes any keywords
# if so, remove those keywords from the similar words list

def compute_similar_words(model,source_word, topn=25):
    similar_words = [source_word]
    try:
        top_words = model.wv.most_similar(source_word, topn=topn)
        length_top = len(top_words)
        i = 0
        while i < length_top:
            #print('i=' + str(i) + ' len(top_words)=' + str(len(top_words)) + 'length_top=' + str(length_top))
            if i >= length_top - 1:
                break
            elif top_words[i][0] in mentions.keys():
                top_words.pop(i)
                length_top -= 1
            else:
                i += 1
        similar_words.extend([val[0] for val in top_words[:5]])
    except KeyError as err:
        print(err.args)
    return similar_words    

def compute_similar_words_and_representative_for_all_tasks(model, topn=25):
    columns = ['word' + str(i - 1) for i in range(1, 7)]
    columns.append('original_tuple')
    df = pd.DataFrame(data=None, columns=columns)
    reps = defaultdict()
    for source_tuple in tuples:
        source_word = (model.wv[source_tuple[0]] + model.wv[source_tuple[1]]) / 2
        similar_words = compute_similar_words(model, model.wv.most_similar(positive=[source_word], topn=1), topn)
        
        df.loc[len(df), ['word0', 'word1', 'word2', 'word3', 'word4', 'word5']] = similar_words
        df.loc[len(df) - 1, ['original_tuple']] = source_tuple[0] + ' ' + source_tuple[1]
        #sum_vec = (model.wv[similar_words[0]]).copy()
        #for i in range(1, 5):
        #    temp = model.wv[similar_words[i]]
        #    sum_vec += temp.copy()
        #rep_vec = sum_vec / 5
        #reps[source_tuple] = model.wv.most_similar(positive=[rep_vec], topn=1)
    #for i in range(1, len(similar_words)):
    #    sec_similar_words = compute_similar_words(model, similar_words[i], topn)
    #    df.loc[len(df)] = sec_similar_words
    df.to_csv('similar_words_task/ethnicity&neighborhood/2018_findings.csv')
    #rep_ser = pd.Series(reps)
    #rep_ser.to_csv('similar_words_task/representative_words/2018_ethnicity&neighborhoods.csv')

In [11]:
compute_similar_words_and_representative_for_all_tasks(model)

C:\Users\ASUS\anaconda3\lib\site-packages\pandas\core\internals\blocks.py:849: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  arr_value = np.array(value)


In [12]:
df

,text
0,nMonday n nOpen at owners’ discretion n nOpe...
1,Born a decade apart Chuck Berry and Mary Tyle...
2,n nThe idea that bigger is better in health ca...
3,With cyclists agitating for more safety measur...
4,A spout of fire that burst from a gas main on ...
...,...
8286,n n n fair After years of bad policy decision...
8287,The governor the mayor the head of the Massa...
8288,The number of students disciplined in schools ...
8289,Hundreds of species are facing extinction due ...


In [13]:
model.wv[tuples[0][0]]

array([ 2.15656646e-02,  1.59779280e-01, -4.29323733e-01,  2.42973551e-01,
       -3.24789882e-01, -3.75209570e-01,  4.67344522e-01,  2.76297808e-01,
       -1.37147069e-01,  3.90075438e-04,  2.26779312e-01, -2.77672917e-01,
       -3.36854935e-01, -1.50636453e-02,  8.87528658e-02,  5.91144599e-02,
        3.72660488e-01, -3.05991650e-01, -4.07284111e-01, -4.30202156e-01,
       -6.54001653e-01,  2.31750786e-01, -5.74461073e-02,  2.05034748e-01,
       -5.94734661e-02, -2.89893313e-03,  8.13017309e-01,  8.11748281e-02,
       -5.97698748e-01,  1.96542427e-01, -1.56266242e-03, -1.30831927e-01,
        3.13586324e-01, -3.06263268e-01, -1.24565795e-01, -2.25767195e-02,
        9.45244789e-01, -1.45498529e-01, -2.76359230e-01, -1.45341724e-01,
       -2.08323196e-01, -1.42506868e-01, -5.62430859e-01, -6.55402467e-02,
        4.54556137e-01, -3.68469983e-01, -4.64076042e-01, -1.60654679e-01,
       -2.03903347e-01,  1.02492236e-01, -2.62166291e-01, -1.68927506e-01,
       -1.75241038e-01,  